# Load data

In [1]:
from datetime import datetime
from collections import defaultdict

from vnpy.trader.database import get_database, DB_TZ
from vnpy.trader.constant import Interval
from vnpy.trader.object import BarData
from vnpy.trader.utility import extract_vt_symbol

db = get_database()

In [2]:
# Load data
vt_symbols = ["BTCUSDT.BINANCE", "ETHUSDT.BINANCE"]

history: dict[datetime, dict[str, BarData]] = defaultdict(dict)

for vt_symbol in vt_symbols:
    symbol, exchange = extract_vt_symbol(vt_symbol)

    bars: list[BarData] = db.load_bar_data(
        symbol=symbol,
        exchange=exchange,
        interval=Interval.MINUTE,
        start=datetime(2024, 8, 1, tzinfo=DB_TZ),
        end=datetime(2024, 9, 10, tzinfo=DB_TZ)
    )

    for bar in bars:
        history[bar.datetime][vt_symbol] = bar
            

# Initialize table

In [3]:
# Choose which table to test
from vnpy_novastrategy import (
    # LiveDataTable as DataTable,           
    BacktestingDataTable as DataTable
)

# Create table
table = DataTable(
    vt_symbols=vt_symbols,
    size=200,
    interval=Interval.MINUTE,
    extra_fields=["active_volume", "active_turnover", "trade_count"]
)


In [ ]:
# Add simple feature
table.add_feature("range", "high_price - low_price")

In [ ]:
# Add feature with parameters
for fast_window, slow_window in [
    (5, 25),
    (10, 50),
    (20, 100)
]:
    table.add_feature(
        name=f"ma_gap_{fast_window}_{slow_window}",
        expression=f"(ts_mean(close_price, {fast_window}) / ts_mean(close_price, {slow_window}) - 1) * 100"
    )

In [ ]:
# Add feature with complex expression
mfi_period = 15
neutral_period = 20

_mfi = f"ta_mfi(high_price, low_price, close_price, volume, {mfi_period})"
_mfi_mean = f"ts_mean({_mfi}, {neutral_period})"
_mfi_std = f"ts_std({_mfi}, {neutral_period})"
_mfi_zscore = f"({_mfi} - {_mfi_mean}) / {_mfi_std}"
expression = f"rank({_mfi_zscore})"
print(expression)

table.add_feature("ranked_mfi", expression)

In [ ]:
# Add feature with complex expressions
mfi_period = 15
neutral_period = 20

_mfi = f"ta_mfi(high_price, low_price, close_price, volume, {mfi_period})"
_mfi_mean = f"ts_mean({_mfi}, {neutral_period})"
_mfi_std = f"ts_std({_mfi}, {neutral_period})"
_mfi_zscore = f"({_mfi} - {_mfi_mean}) / {_mfi_std}"
expression = f"rank({_mfi_zscore})"
print(expression)

table.add_feature("ranked_mfi", expression)

In [ ]:
# Add feature with intermediate variables
short_window = 5
long_window = 20
signal_window = 50

table.add_feature("short_ma", f"ts_mean(close_price, {short_window})")
table.add_feature("long_ma", f"ts_mean(close_price, {long_window})")
table.add_feature("buy_signal_mask", "ts_greater_than(short_ma, long_ma) * ts_greater_than(close_price, short_ma)")
table.add_feature("buy_proportion", f"ts_sum(buy_signal_mask, {signal_window}) / {signal_window}")
table.add_feature("buy_signal_proportion_rank", "rank(buy_proportion)")

In [ ]:
# Add feature with parameters
for rsi_window in [15, 20, 25]:
    for rsi_threshold in range(10, 80, 10):
        name = f"rsi_above_threshold_{rsi_window}_{rsi_threshold}"

        _rsi = f"ta_rsi(close_price, {rsi_window})"
        _rsi_above_threshold = f"ts_greater_than({_rsi}, {rsi_threshold})"
        expression = f"cs_sum({_rsi_above_threshold}) / cs_count(close_price)"

        table.add_feature(name, expression)


In [24]:
# Add feature with cross-sectional zscore
table.add_feature("ma20_ma60", "cs_zscore(ts_mean(close_price, 20) / ts_mean(close_price, 60))")
table.add_feature("ma20_ma120", "cs_zscore(ts_mean(close_price, 20) / ts_mean(close_price, 120))")
table.add_feature("rsi_14", "cs_zscore(ta_rsi(close_price, 14))")

table.add_feature("label", "cs_zscore(ts_delay(close_price, -20) / close_price - 1)")

# For live trading usage

In [ ]:
# Update bars into table    
for dt, bars in history.items():
    table.update_bars(bars)

In [ ]:
# Get latest dataframe
df = table.get_df()
df.tail(20)

# For backtesting usage

In [5]:
# Update entire history into table
table.update_history(list(history.values()))

In [6]:
# Update bars into table    
for dt, bars in history.items():
    table.update_bars(bars)

In [25]:
# Get latest dataframe
df = table.get_df()
df.tail(100)

open_price  high_price  low_price  \
datetime                  vt_symbol                                            
2024-09-09 23:11:00+08:00 BTCUSDT.BINANCE    55130.40    55157.00   55094.30   
                          ETHUSDT.BINANCE     2285.01     2286.74    2282.71   
2024-09-09 23:12:00+08:00 BTCUSDT.BINANCE    55149.10    55155.40   55033.40   
                          ETHUSDT.BINANCE     2286.05     2286.47    2281.07   
2024-09-09 23:13:00+08:00 BTCUSDT.BINANCE    55036.60    55037.40   54864.10   
...                                               ...         ...        ...   
2024-09-09 23:58:00+08:00 ETHUSDT.BINANCE     2293.89     2293.89    2292.00   
2024-09-09 23:59:00+08:00 BTCUSDT.BINANCE    55362.00    55362.00   55340.00   
                          ETHUSDT.BINANCE     2292.01     2292.01    2291.07   
2024-09-10 00:00:00+08:00 BTCUSDT.BINANCE    55360.30    55372.70   55319.40   
                          ETHUSDT.BINANCE     2291.61     2292.33    2290.00   

                                           close_price    volume  \
datetime                  vt_symbol                                
2024-09-09 23:11:00+08:00 BTCUSDT.BINANCE     55149.00   249.498   
                          ETHUSDT.BINANCE      2286.05  2501.342   
2024-09-09 23:12:00+08:00 BTCUSDT.BINANCE     55036.60   480.116   
                          ETHUSDT.BINANCE      2281.32  2318.093   
2024-09-09 23:13:00+08:00 BTCUSDT.BINANCE     54895.90   824.738   
...                                                ...       ...   
2024-09-09 23:58:00+08:00 ETHUSDT.BINANCE      2292.00   544.330   
2024-09-09 23:59:00+08:00 BTCUSDT.BINANCE     55360.30   120.094   
                          ETHUSDT.BINANCE      2291.59   819.934   
2024-09-10 00:00:00+08:00 BTCUSDT.BINANCE     55369.10   138.978   
                          ETHUSDT.BINANCE      2291.78  1457.763   

                                               turnover  open_interest  \
datetime                  vt_symbol                                      
2024-09-09 23:11:00+08:00 BTCUSDT.BINANCE  1.375284e+07              0   
                          ETHUSDT.BINANCE  5.713605e+06              0   
2024-09-09 23:12:00+08:00 BTCUSDT.BINANCE  2.644752e+07              0   
                          ETHUSDT.BINANCE  5.295213e+06              0   
2024-09-09 23:13:00+08:00 BTCUSDT.BINANCE  4.531972e+07              0   
...                                                 ...            ...   
2024-09-09 23:58:00+08:00 ETHUSDT.BINANCE  1.248126e+06              0   
2024-09-09 23:59:00+08:00 BTCUSDT.BINANCE  6.647550e+06              0   
                          ETHUSDT.BINANCE  1.878915e+06              0   
2024-09-10 00:00:00+08:00 BTCUSDT.BINANCE  7.691915e+06              0   
                          ETHUSDT.BINANCE  3.339667e+06              0   

                                           active_volume  active_turnover  \
datetime                  vt_symbol                                         
2024-09-09 23:11:00+08:00 BTCUSDT.BINANCE        105.591     5.820718e+06   
                          ETHUSDT.BINANCE       1285.438     2.936248e+06   
2024-09-09 23:12:00+08:00 BTCUSDT.BINANCE         95.956     5.286836e+06   
                          ETHUSDT.BINANCE        544.013     1.242649e+06   
2024-09-09 23:13:00+08:00 BTCUSDT.BINANCE        250.034     1.373945e+07   
...                                                  ...              ...   
2024-09-09 23:58:00+08:00 ETHUSDT.BINANCE        236.692     5.427377e+05   
2024-09-09 23:59:00+08:00 BTCUSDT.BINANCE         21.304     1.179217e+06   
                          ETHUSDT.BINANCE        288.598     6.613048e+05   
2024-09-10 00:00:00+08:00 BTCUSDT.BINANCE         49.223     2.724305e+06   
                          ETHUSDT.BINANCE        788.398     1.806318e+06   

                                           trade_count  ma20_ma60  ma20_ma120  \
datetime                  vt_symbol                                    